# Transactions data preprocessing

In [169]:
from chain_project.path_linking import summarize_paths, extract_database, merge_subsequences, prefix_span, restore_trades
from chain_project.main import process_transactions

def map_hashes_to_names(database):
    # 生成一个唯一的名称映射
    hash_to_name = {}
    name_counter = 1
    
    for seq in database:
        for hash_value in seq:
            if hash_value not in hash_to_name:
                hash_to_name[hash_value] = f"Name{name_counter}"
                name_counter += 1
    
    # 使用名称映射替换哈希
    mapped_database = []
    for seq in database:
        mapped_seq = [hash_to_name[hash_value] for hash_value in seq]
        mapped_database.append(mapped_seq)
    
    return mapped_database, hash_to_name

file_path='static/bond_2005496_2006_2402.csv'
inst_list = ['长线资本基金孙姣', '国金证券股份严佳', '华创证券有限马延威', '潍坊银行股份王梓涵', '鄂尔多斯银行郭宁', '粤开证券股份周荃', '交通银行股份何嘉隆', '华源证券股份钱淑雯']

data = process_transactions(file_path, inst_list)
json_output = summarize_paths(data)
trade_hashes = extract_database(json_output)

In [170]:
# 原始数据
database = trade_hashes

mapped_database, hash_to_name = map_hashes_to_names(database)

# 打印结果
print("Mapped Database:")
for seq in mapped_database:
    print(seq)

print("\nHash to Name Mapping:")
for hash_value, name in hash_to_name.items():
    print(f"{hash_value}: {name}")


Mapped Database:
['Name1', 'Name2', 'Name3', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8']
['Name1', 'Name2', 'Name3', 'Name9', 'Name10', 'Name8']
['Name1', 'Name2', 'Name3', 'Name9', 'Name11', 'Name8']
['Name1', 'Name2', 'Name3', 'Name9', 'Name7', 'Name8']
['Name1', 'Name2', 'Name3', 'Name9', 'Name12', 'Name13']
['Name3', 'Name9', 'Name10', 'Name14', 'Name15', 'Name16']
['Name17', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8']
['Name3', 'Name9', 'Name10', 'Name14', 'Name18']
['Name3', 'Name9', 'Name11', 'Name19', 'Name18']
['Name9', 'Name10', 'Name19', 'Name18']
['Name20', 'Name6', 'Name7', 'Name8']
['Name21', 'Name11', 'Name19', 'Name18']
['Name22', 'Name10', 'Name8']
['Name22', 'Name11', 'Name8']
['Name1', 'Name14', 'Name18']
['Name1', 'Name23', 'Name24']
['Name25', 'Name11', 'Name8']
['Name1', 'Name23', 'Name26']
['Name27', 'Name23', 'Name26']
['Name21', 'Name11', 'Name8']
['Name1', 'Name28']
['Name1', 'Name8']
['Name27', 'Name28']
['Name27', 'Name8']
['Name17', 'Name29']

Hash to Nam

# 子路径subpath抽取算法设计

现在我有一些交易路径路径，每条交易路径是一笔笔交易组成的，举例，每一笔交易的结构是这样的，将其append为list就是路径的list。

    {
        "seller": "东海证券股份温妍超",
        "buyer": "长线资本基金孙姣",
        "volume": 10,
        "price": 110.36,
        "time_dl": "2023-12-06T16:44:13+08:00",
        "trade_hash": "133a7432cbaead2f7905ae0019b1946bcaf439964d0c2221fdfed64be6d186e2"
      },

其中有一个属性为trade_hash，用于标识交易唯一性。

现在我有很多条这样的交易路径，我的目标是从中抽取出一些小的子路径，请你帮我设计出相关的算法。

小的子路径的认定规则：1. 子路径在多条交易路径中出现。2.最小长度是3，越长越好。

### 子路径生成算法的计算复杂度

上述算法的计算复杂度可以从以下几个方面来分析：

1. **生成子路径的复杂度**：
   - 对于每条交易路径，长度为 \( n \)，生成所有长度大于等于 3 的子路径的复杂度大致为 \( O(n^2) \)。
   - 这是因为在最坏情况下，每个起点可能生成 \( n-2 \) 个子路径（长度从 3 到 \( n \)），而总共有 \( n \) 个起点。

2. **子路径的频率统计**：
   - 假设有 \( m \) 条交易路径，每条路径的长度为 \( n_i \)。
   - 子路径的生成和统计的总复杂度为 \( O\left(\sum_{i=1}^m n_i^2\right) \)，其中 \( n_i \) 是第 \( i \) 条路径的长度。

3. **子路径过滤和排序**：
   - 过滤和排序的复杂度取决于子路径的数量。最坏情况下，子路径的数量可能接近所有路径的总长度的平方，因此排序复杂度为 \( O(k \log k) \)，其中 \( k \) 是所有子路径的总数。

### 总体复杂度
- 因此，整个算法的时间复杂度大致为 \( O\left(\sum_{i=1}^m n_i^2 + k \log k\right) \)。
- 当路径数量 \( m \) 较多，且路径长度较长时，算法的时间复杂度会变得较高。

### 其他的算法

1. **Trie（前缀树）算法**：
   - 可以使用 Trie 数据结构存储所有路径的前缀或子路径。通过在 Trie 中进行深度优先搜索（DFS），可以找到多个路径中重复出现的子路径。
   - 构建 Trie 的复杂度为 \( O\left(\sum_{i=1}^m n_i\right) \)，但查找和过滤重复子路径的复杂度会有所降低。
   - 优点是可以高效地共享公共前缀，减少冗余计算。

2. **后缀数组 + LCP（Longest Common Prefix）算法**：
   - 先将所有交易路径拼接成一个长字符串，使用后缀数组构建所有后缀的排序。然后使用 LCP 数组计算相邻后缀的最长公共前缀，这样可以找到重复子路径。
   - 后缀数组的构建复杂度为 \( O(N \log N) \)，其中 \( N \) 是所有路径长度之和。
   - 这种方法特别适合路径数量非常大且路径之间具有较多相似性的情况。

3. **基于频繁子序列挖掘的算法**：
   - 可以使用频繁子序列挖掘算法（如 PrefixSpan、SPADE 等）来查找多个路径中频繁出现的子路径。这些算法通常用于序列数据的挖掘，能够有效处理子序列在不同路径中的频率。
   - 复杂度依赖于数据的稀疏性和支持度阈值的选择。

### 选择的依据
- 如果你的路径数据量较大，且子路径有较多的重叠，那么 **Trie** 或 **后缀数组 + LCP** 可能会更高效。
- 如果你希望发现更复杂的频繁子路径模式，或者路径数据存在一定的噪声，可以考虑使用 **频繁子序列挖掘算法**。

不同算法的适用性取决于你的具体数据特性和性能需求。

In [171]:
frequent_subsequences = prefix_span(mapped_database, [], min_support=1)
# merged_subsequences = merge_subsequences(frequent_subsequences)

In [172]:
from itertools import combinations
from collections import defaultdict

def generate_subsequences(seq, min_length=1):
    """生成所有可能的子序列"""
    subsequences = set()
    n = len(seq)
    for length in range(min_length, n + 1):
        for start in range(n - length + 1):
            subsequences.add(tuple(seq[start:start + length]))
    return subsequences

def find_frequent_subsequences(sequences, min_occurrences):
    """找到在至少min_occurrences个序列中出现的子序列"""
    subseq_counts = defaultdict(int)
    num_sequences = len(sequences)
    
    # 统计所有子序列的出现次数
    for seq in sequences:
        subsequences = generate_subsequences(seq, min_length=3)  # 子序列最小长度为3
        unique_subsequences = set(subsequences)  # 去重
        for subseq in unique_subsequences:
            subseq_counts[subseq] += 1

    # 过滤出在至少min_occurrences个序列中出现的子序列
    frequent_subsequences = {subseq: count for subseq, count in subseq_counts.items() if count >= min_occurrences}
    
    return frequent_subsequences

# 示例数据
sequences = [
    ['Name1', 'Name2', 'Name3', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name10', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name11', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name7', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name12', 'Name13'],
    ['Name3', 'Name9', 'Name10', 'Name14', 'Name15', 'Name16'],
    ['Name17', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8'],
    ['Name3', 'Name9', 'Name10', 'Name14', 'Name18'],
    ['Name3', 'Name9', 'Name11', 'Name19', 'Name18'],
    ['Name9', 'Name10', 'Name19', 'Name18'],
    ['Name20', 'Name6', 'Name7', 'Name8'],
    ['Name21', 'Name11', 'Name19', 'Name18'],
    ['Name22', 'Name10', 'Name8'],
    ['Name22', 'Name11', 'Name8'],
    ['Name1', 'Name14', 'Name18'],
    ['Name1', 'Name23', 'Name24'],
    ['Name25', 'Name11', 'Name8'],
    ['Name1', 'Name23', 'Name26'],
    ['Name27', 'Name23', 'Name26'],
    ['Name21', 'Name11', 'Name8']
]

# 查找至少出现3次的频繁子序列
min_occurrences = 2
frequent_subsequences = find_frequent_subsequences(sequences, min_occurrences)
print(f"Frequent subsequences appearing in at least {min_occurrences} sequences:")
for subseq, count in frequent_subsequences.items():
    print(f"Subsequence: {subseq}, Count: {count}")


Frequent subsequences appearing in at least 2 sequences:
Subsequence: ('Name5', 'Name6', 'Name7', 'Name8'), Count: 2
Subsequence: ('Name4', 'Name5', 'Name6', 'Name7', 'Name8'), Count: 2
Subsequence: ('Name4', 'Name5', 'Name6', 'Name7'), Count: 2
Subsequence: ('Name4', 'Name5', 'Name6'), Count: 2
Subsequence: ('Name5', 'Name6', 'Name7'), Count: 2
Subsequence: ('Name6', 'Name7', 'Name8'), Count: 3
Subsequence: ('Name1', 'Name2', 'Name3'), Count: 5
Subsequence: ('Name2', 'Name3', 'Name9'), Count: 4
Subsequence: ('Name3', 'Name9', 'Name10'), Count: 3
Subsequence: ('Name1', 'Name2', 'Name3', 'Name9'), Count: 4
Subsequence: ('Name3', 'Name9', 'Name11'), Count: 2
Subsequence: ('Name3', 'Name9', 'Name10', 'Name14'), Count: 2
Subsequence: ('Name9', 'Name10', 'Name14'), Count: 2
Subsequence: ('Name11', 'Name19', 'Name18'), Count: 2


In [173]:
# TODO: 现在有了很多的子序列，但是子序列之中有一些是另一些的子集，这样会导致重复计算，需要对子序列进行去重。
# 去重的规则是 如果两个子序列，其中一个序列是另一个的子集
# 1. 他们的重复次数相同，那么就将他们合并成一个子序列（保留更长的那一个）
# 2. 重复次数不同，其中短的要更长一点，那边保留短的那个，否则就保留短的那一个

In [174]:
from itertools import combinations
from collections import defaultdict

def generate_subsequences(seq, min_length=1):
    """生成所有可能的子序列"""
    subsequences = set()
    n = len(seq)
    for length in range(min_length, n + 1):
        for start in range(n - length + 1):
            subsequences.add(tuple(seq[start:start + length]))
    return subsequences

def find_frequent_subsequences(sequences, min_occurrences):
    """找到在至少min_occurrences个序列中出现的子序列"""
    subseq_counts = defaultdict(int)
    num_sequences = len(sequences)
    
    # 统计所有子序列的出现次数
    for seq in sequences:
        subsequences = generate_subsequences(seq, min_length=3)  # 子序列最小长度为3
        unique_subsequences = set(subsequences)  # 去重
        for subseq in unique_subsequences:
            subseq_counts[subseq] += 1

    # 过滤出在至少min_occurrences个序列中出现的子序列
    frequent_subsequences = {subseq: count for subseq, count in subseq_counts.items() if count >= min_occurrences}
    
    return frequent_subsequences

def remove_subset_duplicates(subsequences):
    """去除子集重复的子序列"""
    sorted_subsequences = sorted(subsequences.items(), key=lambda x: (-len(x[0]), -x[1]))  # 按长度和计数排序
    unique_subsequences = {}
    
    for subseq, count in sorted_subsequences:
        is_subset = False
        for other_subseq in list(unique_subsequences.keys()):
            if set(subseq).issubset(set(other_subseq)):
                is_subset = True
                if unique_subsequences[other_subseq] == count:
                    # 如果两个子序列出现次数相同，保留更长的那一个
                    if len(subseq) > len(other_subseq):
                        del unique_subsequences[other_subseq]
                        unique_subsequences[subseq] = count
                break
        if not is_subset:
            unique_subsequences[subseq] = count
    
    return unique_subsequences

# 示例数据
sequences = [
    ['Name1', 'Name2', 'Name3', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name10', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name11', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name7', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name12', 'Name13'],
    ['Name3', 'Name9', 'Name10', 'Name14', 'Name15', 'Name16'],
    ['Name17', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8'],
    ['Name3', 'Name9', 'Name10', 'Name14', 'Name18'],
    ['Name3', 'Name9', 'Name11', 'Name19', 'Name18'],
    ['Name9', 'Name10', 'Name19', 'Name18'],
    ['Name20', 'Name6', 'Name7', 'Name8'],
    ['Name21', 'Name11', 'Name19', 'Name18'],
    ['Name22', 'Name10', 'Name8'],
    ['Name22', 'Name11', 'Name8'],
    ['Name1', 'Name14', 'Name18'],
    ['Name1', 'Name23', 'Name24'],
    ['Name25', 'Name11', 'Name8'],
    ['Name1', 'Name23', 'Name26'],
    ['Name27', 'Name23', 'Name26'],
    ['Name21', 'Name11', 'Name8']
]

# 查找至少出现2次的频繁子序列
min_occurrences = 2
frequent_subsequences = find_frequent_subsequences(sequences, min_occurrences)

# 去除子集重复
unique_subsequences = remove_subset_duplicates(frequent_subsequences)

print(f"Unique frequent subsequences appearing in at least {min_occurrences} sequences:")
for subseq, count in unique_subsequences.items():
    print(f"Subsequence: {subseq}, Count: {count}")


Unique frequent subsequences appearing in at least 2 sequences:
Subsequence: ('Name4', 'Name5', 'Name6', 'Name7', 'Name8'), Count: 2
Subsequence: ('Name1', 'Name2', 'Name3', 'Name9'), Count: 4
Subsequence: ('Name3', 'Name9', 'Name10', 'Name14'), Count: 2
Subsequence: ('Name3', 'Name9', 'Name11'), Count: 2
Subsequence: ('Name11', 'Name19', 'Name18'), Count: 2


In [175]:
# Frequent subsequences appearing in at least 2 sequences:
# Subsequence: ('Name5', 'Name6', 'Name7', 'Name8'), Count: 2
# Subsequence: ('Name4', 'Name5', 'Name6', 'Name7', 'Name8'), Count: 2
# Subsequence: ('Name5', 'Name6', 'Name7'), Count: 2
# Subsequence: ('Name6', 'Name7', 'Name8'), Count: 3
# Subsequence: ('Name4', 'Name5', 'Name6', 'Name7'), Count: 2
# Subsequence: ('Name1', 'Name2', 'Name3'), Count: 5
# Subsequence: ('Name4', 'Name5', 'Name6'), Count: 2
# Subsequence: ('Name3', 'Name9', 'Name10'), Count: 3
# Subsequence: ('Name2', 'Name3', 'Name9'), Count: 4
# Subsequence: ('Name1', 'Name2', 'Name3', 'Name9'), Count: 4
# Subsequence: ('Name3', 'Name9', 'Name11'), Count: 2
# Subsequence: ('Name9', 'Name10', 'Name14'), Count: 2
# Subsequence: ('Name3', 'Name9', 'Name10', 'Name14'), Count: 2
# Subsequence: ('Name11', 'Name19', 'Name18'), Count: 2

In [176]:
# 得到了最小的子序列，接下来需要将转换到原本的链路内部，方法是从每条链路内部查找最小的子序列，然后将其替换到原链路上。

{
  "nodes":[
    {
      "name": 'Name1',  #随机生成一个
      "content": 'Name1'
    },
    {
      "name": 'subsequence1',  #随机生成一个
      "content": ['Name3', 'Name9', 'Name10']
    },
    {
      "name": 'Name8',  #随机生成一个
      "content": 'Name8'
    },
    {
      "name": 'subsequence' + '2',  #随机生成一个
      "content": ['Name1', 'Name2', 'Name3']
    }],
  "flows":[
    {
      "thru": [
        "Name1",
        "Name2",
        "subsequence1",
        "Name8"
      ],
      "value": 1
    },
  ]

}

{'nodes': [{'name': 'Name1', 'content': 'Name1'},
  {'name': 'subsequence1', 'content': ['Name3', 'Name9', 'Name10']},
  {'name': 'Name8', 'content': 'Name8'},
  {'name': 'subsequence2', 'content': ['Name1', 'Name2', 'Name3']}],
 'flows': [{'thru': ['Name1', 'Name2', 'subsequence1', 'Name8'], 'value': 1}]}

In [177]:
import random
import string

def generate_random_name(prefix=''):
    """生成随机名称"""
    return prefix + ''.join(random.choices(string.ascii_letters + string.digits, k=5))

def map_subsequences_to_nodes(subsequences):
    """将最小子序列映射到节点"""
    node_map = {}
    node_id = 1
    for subseq in subsequences:
        node_name = generate_random_name(prefix='subsequence')
        node_map[subseq] = node_name
    return node_map

def identify_individual_transactions(sequences, subsequences):
    """识别单独交易"""
    all_transactions = set(transaction for seq in sequences for transaction in seq)
    subseq_transactions = set(transaction for subseq in subsequences for transaction in subseq)
    individual_transactions = all_transactions - subseq_transactions
    return individual_transactions

def replace_subsequences_in_sequences(sequences, subsequences_map):
    """用节点名称替换原链路中的最小子序列"""
    replaced_sequences = []
    for seq in sequences:
        replaced_seq = []
        i = 0
        while i < len(seq):
            matched = False
            for length in range(5, 2, -1):  # 从最长到最短检查
                if i + length <= len(seq):
                    sub = tuple(seq[i:i + length])
                    if sub in subsequences_map:
                        replaced_seq.append(subsequences_map[sub])
                        i += length
                        matched = True
                        break
            if not matched:
                replaced_seq.append(seq[i])
                i += 1
        replaced_sequences.append(replaced_seq)
    return replaced_sequences

def generate_flows(sequences):
    """生成流动数据"""
    flows = []
    for seq in sequences:
        flow = {"thru": seq, "value": 1}
        flows.append(flow)
    return flows

# 最小子序列
min_subsequences = [
    ('Name4', 'Name5', 'Name6', 'Name7', 'Name8'),
    ('Name1', 'Name2', 'Name3', 'Name9'),
    ('Name3', 'Name9', 'Name10', 'Name14'),
    ('Name3', 'Name9', 'Name11'),
    ('Name11', 'Name19', 'Name18')
]

# 原始数据
sequences = [
    ['Name1', 'Name2', 'Name3', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name10', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name11', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name7', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name12', 'Name13'],
    ['Name3', 'Name9', 'Name10', 'Name14', 'Name15', 'Name16'],
    ['Name17', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8'],
    ['Name3', 'Name9', 'Name10', 'Name14', 'Name18'],
    ['Name3', 'Name9', 'Name11', 'Name19', 'Name18'],
    ['Name9', 'Name10', 'Name19', 'Name18'],
    ['Name20', 'Name6', 'Name7', 'Name8'],
    ['Name21', 'Name11', 'Name19', 'Name18'],
    ['Name22', 'Name10', 'Name8'],
    ['Name22', 'Name11', 'Name8'],
    ['Name1', 'Name14', 'Name18'],
    ['Name1', 'Name23', 'Name24'],
    ['Name25', 'Name11', 'Name8'],
    ['Name1', 'Name23', 'Name26'],
    ['Name27', 'Name23', 'Name26'],
    ['Name21', 'Name11', 'Name8']
]

# 生成节点映射
node_map = map_subsequences_to_nodes(min_subsequences)

# 识别单独交易
individual_transactions = identify_individual_transactions(sequences, min_subsequences)

# 将单独交易也加入到节点映射中
for transaction in individual_transactions:
    node_map[tuple([transaction])] = generate_random_name(prefix='transaction')

# 将原始数据中的子序列替换为节点名称
replaced_sequences = replace_subsequences_in_sequences(sequences, node_map)

# 生成流动数据
flows = generate_flows(replaced_sequences)

# 生成最终结果
result = {
    "nodes": [
        {"name": node_map[('Name4', 'Name5', 'Name6', 'Name7', 'Name8')], "disp": ['Name4', 'Name5', 'Name6', 'Name7', 'Name8']},
        {"name": node_map[('Name1', 'Name2', 'Name3', 'Name9')], "disp": ['Name1', 'Name2', 'Name3', 'Name9']},
        {"name": node_map[('Name3', 'Name9', 'Name10', 'Name14')], "disp": ['Name3', 'Name9', 'Name10', 'Name14']},
        {"name": node_map[('Name3', 'Name9', 'Name11')], "disp": ['Name3', 'Name9', 'Name11']},
        {"name": node_map[('Name11', 'Name19', 'Name18')], "disp": ['Name11', 'Name19', 'Name18']},
        *[
            {"name": node_map[tuple([transaction])], "disp": transaction}
            for transaction in individual_transactions
        ]
    ],
    "flows": flows
}

print(result)


{'nodes': [{'name': 'subsequencelnduE', 'disp': ['Name4', 'Name5', 'Name6', 'Name7', 'Name8']}, {'name': 'subsequenceNInWL', 'disp': ['Name1', 'Name2', 'Name3', 'Name9']}, {'name': 'subsequenceQmS9H', 'disp': ['Name3', 'Name9', 'Name10', 'Name14']}, {'name': 'subsequenceGzK12', 'disp': ['Name3', 'Name9', 'Name11']}, {'name': 'subsequenceI6Mt8', 'disp': ['Name11', 'Name19', 'Name18']}, {'name': 'transactionEHd81', 'disp': 'Name17'}, {'name': 'transactionteLD4', 'disp': 'Name24'}, {'name': 'transactionEI2uP', 'disp': 'Name25'}, {'name': 'transactionIMDKj', 'disp': 'Name15'}, {'name': 'transactionAXVzH', 'disp': 'Name26'}, {'name': 'transactionAW2No', 'disp': 'Name16'}, {'name': 'transaction0Ryge', 'disp': 'Name12'}, {'name': 'transactionVyQNy', 'disp': 'Name22'}, {'name': 'transactionPSFNz', 'disp': 'Name27'}, {'name': 'transactionGd2cq', 'disp': 'Name20'}, {'name': 'transactionKQn6K', 'disp': 'Name23'}, {'name': 'transaction4LPif', 'disp': 'Name21'}, {'name': 'transactionA4NQf', 'disp':

In [178]:
# import json
# # 将结果保存为 JSON 文件
# with open('data.json', 'w') as f:
#     json.dump(result, f, indent=4)

# print("结果已保存为 result.json")

In [179]:
import random
import string
import json

def generate_random_name(prefix=''):
    """生成随机名称"""
    return prefix + ''.join(random.choices(string.ascii_letters + string.digits, k=5))

def map_subsequences_to_nodes(subsequences):
    """将最小子序列映射到节点"""
    node_map = {}
    node_id = 1
    for subseq in subsequences:
        node_name = generate_random_name(prefix='subsequence')
        node_map[subseq] = node_name
    return node_map

def identify_individual_transactions(sequences, subsequences):
    """识别单独交易"""
    all_transactions = set(transaction for seq in sequences for transaction in seq)
    subseq_transactions = set(transaction for subseq in subsequences for transaction in subseq)
    individual_transactions = all_transactions - subseq_transactions
    return individual_transactions

def replace_subsequences_in_sequences(sequences, subsequences_map):
    """用节点名称替换原链路中的最小子序列"""
    replaced_sequences = []
    for seq in sequences:
        replaced_seq = []
        i = 0
        while i < len(seq):
            matched = False
            for length in range(5, 2, -1):  # 从最长到最短检查
                if i + length <= len(seq):
                    sub = tuple(seq[i:i + length])
                    if sub in subsequences_map:
                        replaced_seq.append(subsequences_map[sub])
                        i += length
                        matched = True
                        break
            if not matched:
                replaced_seq.append(seq[i])
                i += 1
        replaced_sequences.append(replaced_seq)
    return replaced_sequences

def generate_flows(sequences, node_map):
    """生成流动数据"""
    flows = []
    for seq in sequences:
        flow = {"thru": [node_map.get(item, item) for item in seq], "value": 1}
        flows.append(flow)
    return flows

def format_nodes(node_map, individual_transactions):
    """格式化节点"""
    nodes = []
    for subseq, name in node_map.items():
        nodes.append({"disp": ', '.join(subseq), "name": name})
    for transaction in individual_transactions:
        transaction_name = generate_random_name(prefix='transaction')
        nodes.append({"disp": transaction, "name": transaction_name})
    return nodes

def add_missing_nodes(flows, nodes):
    """检查并补齐缺失的节点"""
    nodes_set = set(node["name"] for node in nodes)
    missing_nodes = set()
    
    for flow in flows:
        for item in flow["thru"]:
            if item not in nodes_set:
                missing_nodes.add(item)

    for missing in missing_nodes:
        node_name = generate_random_name(prefix='transaction')
        nodes.append({"disp": missing, "name": node_name})
    
    return nodes

# 最小子序列
min_subsequences = [
    ('Name4', 'Name5', 'Name6', 'Name7', 'Name8'),
    ('Name1', 'Name2', 'Name3', 'Name9'),
    ('Name3', 'Name9', 'Name10', 'Name14'),
    ('Name3', 'Name9', 'Name11'),
    ('Name11', 'Name19', 'Name18')
]

# 原始数据
sequences = [
    ['Name1', 'Name2', 'Name3', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name10', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name11', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name7', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name12', 'Name13'],
    ['Name3', 'Name9', 'Name10', 'Name14', 'Name15', 'Name16'],
    ['Name17', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8'],
    ['Name3', 'Name9', 'Name10', 'Name14', 'Name18'],
    ['Name3', 'Name9', 'Name11', 'Name19', 'Name18'],
    ['Name9', 'Name10', 'Name19', 'Name18'],
    ['Name20', 'Name6', 'Name7', 'Name8'],
    ['Name21', 'Name11', 'Name19', 'Name18'],
    ['Name22', 'Name10', 'Name8'],
    ['Name22', 'Name11', 'Name8'],
    ['Name1', 'Name14', 'Name18'],
    ['Name1', 'Name23', 'Name24'],
    ['Name25', 'Name11', 'Name8'],
    ['Name1', 'Name23', 'Name26'],
    ['Name27', 'Name23', 'Name26'],
    ['Name21', 'Name11', 'Name8']
]

# 生成节点映射
node_map = map_subsequences_to_nodes(min_subsequences)

# 识别单独交易
individual_transactions = identify_individual_transactions(sequences, min_subsequences)

# 将单独交易也加入到节点映射中
for transaction in individual_transactions:
    node_map[tuple([transaction])] = generate_random_name(prefix='transaction')

# 将原始数据中的子序列替换为节点名称
replaced_sequences = replace_subsequences_in_sequences(sequences, node_map)

# 生成流动数据
flows = generate_flows(replaced_sequences, node_map)

# 格式化节点
nodes = format_nodes(node_map, individual_transactions)

# 检查并补齐缺失的节点
nodes = add_missing_nodes(flows, nodes)

# 生成最终结果
result = {
    "Data source": "[Robert J. MacG. Dawson](http://www.amstat.org/publications/jse/v3n3/datasets.dawson.html)",
    "nodes": nodes,
    "flows": flows
}




In [180]:
import random
import string

def generate_random_name(prefix=''):
    """生成随机名称"""
    return prefix + ''.join(random.choices(string.ascii_letters + string.digits, k=5))

def map_subsequences_to_nodes(subsequences):
    """将最小子序列映射到节点"""
    node_map = {}
    node_id = 1
    for subseq in subsequences:
        node_name = generate_random_name(prefix='subsequence')
        node_map[subseq] = node_name
    return node_map

def identify_individual_transactions(sequences, subsequences):
    """识别单独交易"""
    all_transactions = set(transaction for seq in sequences for transaction in seq)
    subseq_transactions = set(transaction for subseq in subsequences for transaction in subseq)
    individual_transactions = all_transactions
    return individual_transactions

def replace_subsequences_in_sequences(sequences, subsequences_map):
    """用节点名称替换原链路中的最小子序列"""
    replaced_sequences = []
    for seq in sequences:
        replaced_seq = []
        i = 0
        while i < len(seq):
            matched = False
            for length in range(5, 2, -1):  # 从最长到最短检查
                if i + length <= len(seq):
                    sub = tuple(seq[i:i + length])
                    if sub in subsequences_map:
                        replaced_seq.append(subsequences_map[sub])
                        i += length
                        matched = True
                        break
            if not matched:
                replaced_seq.append(node_map[seq[i]])
                i += 1
        replaced_sequences.append(replaced_seq)
    return replaced_sequences

def generate_flows(sequences, node_map):
    """生成流动数据"""
    flows = []
    for seq in sequences:
        flow = {"thru": [node_map.get(x, x) for x in seq], "value": 1}
        flows.append(flow)
    return flows

def format_nodes(node_map, individual_transactions):
    """格式化节点"""
    nodes = []
    for subseq, name in node_map.items():
        nodes.append({"disp": subseq, "name": name})
    for transaction in individual_transactions:
        transaction_name = generate_random_name(prefix='transaction')
        nodes.append({"disp": transaction, "name": transaction_name})
        node_map[transaction] = transaction_name  # Ensure the transaction is included in the map
    return nodes

# 最小子序列
min_subsequences = [
    ('Name4', 'Name5', 'Name6', 'Name7', 'Name8'),
    ('Name1', 'Name2', 'Name3', 'Name9'),
    ('Name3', 'Name9', 'Name10', 'Name14'),
    ('Name3', 'Name9', 'Name11'),
    ('Name11', 'Name19', 'Name18')
]

# 原始数据
sequences = [
    ['Name1', 'Name2', 'Name3', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name10', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name11', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name7', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name12', 'Name13'],
    ['Name3', 'Name9', 'Name10', 'Name14', 'Name15', 'Name16'],
    ['Name17', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8'],
    ['Name3', 'Name9', 'Name10', 'Name14', 'Name18'],
    ['Name3', 'Name9', 'Name11', 'Name19', 'Name18'],
    ['Name9', 'Name10', 'Name19', 'Name18'],
    ['Name20', 'Name6', 'Name7', 'Name8'],
    ['Name21', 'Name11', 'Name19', 'Name18'],
    ['Name22', 'Name10', 'Name8'],
    ['Name22', 'Name11', 'Name8'],
    ['Name1', 'Name14', 'Name18'],
    ['Name1', 'Name23', 'Name24'],
    ['Name25', 'Name11', 'Name8'],
    ['Name1', 'Name23', 'Name26'],
    ['Name27', 'Name23', 'Name26'],
    ['Name21', 'Name11', 'Name8']
]

# 生成节点映射
node_map = map_subsequences_to_nodes(min_subsequences)

# 识别单独交易
individual_transactions = identify_individual_transactions(sequences, min_subsequences)

# 生成每笔交易的节点映射
for transaction in individual_transactions:
    node_map[transaction] = generate_random_name(prefix='transaction')

# 将原始数据中的子序列替换为节点名称
replaced_sequences = replace_subsequences_in_sequences(sequences, node_map)

# 生成流动数据
flows = generate_flows(replaced_sequences, node_map)

# 格式化节点
nodes = format_nodes(node_map, individual_transactions)

# 生成最终结果
result = {
    "nodes": nodes,
    "flows": flows
}

print(result)

# 保存为 JSON 文件
with open('result_with_check.json', 'w') as f:
    json.dump(result, f, indent=4)

{'nodes': [{'disp': ('Name4', 'Name5', 'Name6', 'Name7', 'Name8'), 'name': 'subsequenceVL5uE'}, {'disp': ('Name1', 'Name2', 'Name3', 'Name9'), 'name': 'subsequencenXHbx'}, {'disp': ('Name3', 'Name9', 'Name10', 'Name14'), 'name': 'subsequencezRnK3'}, {'disp': ('Name3', 'Name9', 'Name11'), 'name': 'subsequence9vJe8'}, {'disp': ('Name11', 'Name19', 'Name18'), 'name': 'subsequence805IX'}, {'disp': 'Name7', 'name': 'transaction2uNYu'}, {'disp': 'Name25', 'name': 'transactiondXBzU'}, {'disp': 'Name16', 'name': 'transaction52ZFc'}, {'disp': 'Name14', 'name': 'transactionOs82d'}, {'disp': 'Name21', 'name': 'transactionRbHON'}, {'disp': 'Name19', 'name': 'transactionnNRU3'}, {'disp': 'Name9', 'name': 'transactionOdZC4'}, {'disp': 'Name24', 'name': 'transactionm4Ie4'}, {'disp': 'Name4', 'name': 'transactionX1gbo'}, {'disp': 'Name1', 'name': 'transactionimO75'}, {'disp': 'Name12', 'name': 'transactionkj8lg'}, {'disp': 'Name23', 'name': 'transactionNpyIB'}, {'disp': 'Name13', 'name': 'transactionj

In [181]:
# 最小子序列
min_subsequences = [
    ('Name4', 'Name5', 'Name6', 'Name7', 'Name8'),
    ('Name1', 'Name2', 'Name3', 'Name9'),
    ('Name3', 'Name9', 'Name10', 'Name14'),
    ('Name3', 'Name9', 'Name11'),
    ('Name11', 'Name19', 'Name18')
]

# 原始数据
sequences = [
    ['Name1', 'Name2', 'Name3', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name10', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name11', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name7', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name12', 'Name13'],
    ['Name3', 'Name9', 'Name10', 'Name14', 'Name15', 'Name16'],
    ['Name17', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8'],
    ['Name3', 'Name9', 'Name10', 'Name14', 'Name18'],
    ['Name3', 'Name9', 'Name11', 'Name19', 'Name18'],
    ['Name9', 'Name10', 'Name19', 'Name18'],
    ['Name20', 'Name6', 'Name7', 'Name8'],
    ['Name21', 'Name11', 'Name19', 'Name18'],
    ['Name22', 'Name10', 'Name8'],
    ['Name22', 'Name11', 'Name8'],
    ['Name1', 'Name14', 'Name18'],
    ['Name1', 'Name23', 'Name24'],
    ['Name25', 'Name11', 'Name8'],
    ['Name1', 'Name23', 'Name26'],
    ['Name27', 'Name23', 'Name26'],
    ['Name21', 'Name11', 'Name8']
]

#TODO: 应该是在原始数据中的每条sequence中将最小子序列的元素去掉，得到一段一段的新的序列，再对新序列进行上述的一波操作。
# 中间被去掉的元素应该找个东西当做占位符 前后被剪断的部分和中间的占位符用一个list包起来，这样再下一轮的最小子序列迭代过程中就可以更细粒度了。
def identify_individual_transactions(sequences, subsequences):
    """识别单独交易"""
    all_transactions = set(transaction for seq in sequences for transaction in seq)
    subseq_transactions = set(transaction for subseq in subsequences for transaction in subseq)
    individual_transactions = all_transactions
    return individual_transactions

In [182]:
import random
import string
from collections import Counter

def generate_random_name(prefix=''):
    """生成随机名称"""
    return prefix + ''.join(random.choices(string.ascii_letters + string.digits, k=5))

def find_frequent_subsequences(sequences, length, removed_subsequences):
    """查找频繁出现的子序列"""
    subseq_count = Counter()
    for seq in sequences:
        # 过滤掉已经被移除的子序列
        filtered_seq = [x for x in seq if x not in removed_subsequences]
        for i in range(len(filtered_seq) - length + 1):
            sub = tuple(filtered_seq[i:i + length])
            if not any(item.startswith('<removed_') for item in sub):
                subseq_count[sub] += 1
    return [subseq for subseq, count in subseq_count.items() if count > 1]

def replace_subsequences(seq, subsequences_map, placeholder):
    """将子序列替换为占位符"""
    replaced_seq = []
    i = 0
    while i < len(seq):
        matched = False
        for length in range(len(subsequences_map), 1, -1):  # 从最长到最短检查
            if i + length <= len(seq):
                sub = tuple(seq[i:i + length])
                if sub in subsequences_map:
                    replaced_seq.append(subsequences_map[sub])
                    i += length
                    matched = True
                    break
        if not matched:
            replaced_seq.append(seq[i])
            i += 1
    return replaced_seq

def process_sequences(sequences, lengths, placeholders):
    """逐步处理序列"""
    processed_sequences = sequences[:]
    all_node_maps = {}
    removed_subsequences = set()
    
    for length, placeholder in zip(lengths, placeholders):
        # 查找频繁子序列（排除已经被替换的子序列）
        subsequences = find_frequent_subsequences(processed_sequences, length, removed_subsequences)
        
        # 生成节点映射
        node_map = {subseq: generate_random_name(prefix=placeholder) for subseq in subsequences}
        all_node_maps.update(node_map)
        
        # 更新已移除的子序列集合
        removed_subsequences.update(node_map.keys())
        print('removed_subsequences', removed_subsequences)
        
        # 替换子序列
        processed_sequences = [replace_subsequences(seq, node_map, placeholder) for seq in processed_sequences]
    
    return processed_sequences, all_node_maps

def generate_flows1(sequences, node_map):
    """生成流动数据"""
    flows = []
    for seq in sequences:
        flow = {"thru": [node_map.get(x, x) for x in seq if x not in node_map], "value": 1}
        flows.append(flow)
    return flows

def generate_flows(sequences, node_map):
    """生成流动数据"""
    flows = []
    
    # 遍历每条序列
    for seq in sequences:

        # 创建流动数据的列表
        updated_seq = []
        
        # 遍历序列中的每个项
        for x in seq:
            
            if x not in node_map:
                # 如果该项不在节点映射中，则生成新的名称并更新节点映射
                new_name = generate_random_name(prefix='transaction')
                node_map[x] = new_name
            
            # 使用节点名称或原始名称
            updated_seq.append(node_map[x])
        
        # 添加到流动数据列表
        flow = {"thru": updated_seq, "value": 1}
        flows.append(flow)
    print('node_map', node_map)
    return flows

def format_nodes(node_map):
    """格式化节点"""
    nodes = []
    for subseq, name in node_map.items():
        nodes.append({"disp": subseq, "name": name})
    return nodes

def separate_removed_subsequences(sequences, removed_subsequences):
    """将已经移除的子序列单独处理"""
    individual_nodes = {}
    for subseq in removed_subsequences:
        name = generate_random_name(prefix='<removed>')
        individual_nodes[tuple(subseq)] = name
    return individual_nodes

# 原始数据
sequences = [
    ['Name1', 'Name2', 'Name3', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name10', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name11', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name7', 'Name8'],
    ['Name1', 'Name2', 'Name3', 'Name9', 'Name12', 'Name13'],
    ['Name3', 'Name9', 'Name10', 'Name14', 'Name15', 'Name16'],
    ['Name17', 'Name4', 'Name5', 'Name6', 'Name7', 'Name8'],
    ['Name3', 'Name9', 'Name10', 'Name14', 'Name18'],
    ['Name3', 'Name9', 'Name11', 'Name19', 'Name18'],
    ['Name9', 'Name10', 'Name19', 'Name18'],
    ['Name20', 'Name6', 'Name7', 'Name8'],
    ['Name21', 'Name11', 'Name19', 'Name18'],
    ['Name22', 'Name10', 'Name8'],
    ['Name22', 'Name11', 'Name8'],
    ['Name1', 'Name14', 'Name18'],
    ['Name1', 'Name23', 'Name24'],
    ['Name25', 'Name11', 'Name8'],
    ['Name1', 'Name23', 'Name26'],
    ['Name27', 'Name23', 'Name26'],
    ['Name21', 'Name11', 'Name8']
]

# 定义最小子序列长度和占位符
lengths = [4, 3, 2]
placeholders = ['<removed_4>', '<removed_3>', '<removed_2>']

# 逐步处理序列
processed_sequences, node_map = process_sequences(sequences, lengths, placeholders)

# 生成占位符的单独节点
removed_subsequences = {subseq for subseq, name in node_map.items() if '<removed>' in name}
individual_nodes = separate_removed_subsequences(sequences, removed_subsequences)
node_map.update(individual_nodes)

# 生成流动数据
flows = generate_flows(processed_sequences, node_map)

# 格式化节点
nodes = format_nodes(node_map)

# 生成最终结果
result = {
    "Data source": "[Robert J. MacG. Dawson](http://www.amstat.org/publications/jse/v3n3/datasets.dawson.html)",
    "nodes": nodes,
    "flows": flows
}

print(result)

# 保存为 JSON 文件
with open('result_with_check11.json', 'w') as f:
    json.dump(result, f, indent=4)

removed_subsequences {('Name3', 'Name9', 'Name10', 'Name14'), ('Name4', 'Name5', 'Name6', 'Name7'), ('Name5', 'Name6', 'Name7', 'Name8'), ('Name1', 'Name2', 'Name3', 'Name9')}
removed_subsequences {('Name5', 'Name6', 'Name7', 'Name8'), ('Name11', 'Name19', 'Name18'), ('Name3', 'Name9', 'Name10', 'Name14'), ('Name4', 'Name5', 'Name6', 'Name7'), ('Name1', 'Name2', 'Name3', 'Name9')}
removed_subsequences {('Name5', 'Name6', 'Name7', 'Name8'), ('Name21', 'Name11'), ('Name11', 'Name19', 'Name18'), ('Name3', 'Name9', 'Name10', 'Name14'), ('Name4', 'Name5', 'Name6', 'Name7'), ('Name11', 'Name8'), ('Name19', 'Name18'), ('Name10', 'Name8'), ('Name11', 'Name19'), ('Name23', 'Name26'), ('Name7', 'Name8'), ('Name1', 'Name23'), ('Name1', 'Name2', 'Name3', 'Name9')}
node_map {('Name4', 'Name5', 'Name6', 'Name7'): '<removed_4>ERt4d', ('Name5', 'Name6', 'Name7', 'Name8'): '<removed_4>XPWmM', ('Name1', 'Name2', 'Name3', 'Name9'): '<removed_4>hOrdY', ('Name3', 'Name9', 'Name10', 'Name14'): '<removed_4>R